In [1]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import DecomposeResult, seasonal_decompose
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

pd.options.display.width = None
warnings.filterwarnings('ignore')
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)  # format numeric outputs

# Reading data

In [2]:
df_2019 = pd.read_excel("E:\\visual stodio code\\GP\\Data.xlsx", sheet_name='2019')
df_2019['order_date'] = pd.to_datetime(df_2019['order_date'], format='%m/%d/%y')

df_2020 = pd.read_excel("E:\\visual stodio code\\GP\\Data.xlsx", sheet_name='2020')
df_2020['order_date'] = pd.to_datetime(df_2020['order_date'], format='%m/%d/%y')

df_2021 = pd.concat(pd.read_excel("E:\\visual stodio code\\GP\\Data.xlsx", sheet_name=['2021 - 1', '2021 - 2', '2021 - 3']), ignore_index=True)
df_2021['order_date'] = pd.to_datetime(df_2021['order_date'], format='%m/%d/%y')

In [3]:
df_2019.head()

,order_date,retailer_id,product_id,quantities_sold
0,2019-10-19,63,8,3.000
1,2019-12-14,63,8,4.000
2,2019-12-21,63,8,3.000
3,2019-12-07,67,8,2.000
4,2019-12-28,67,8,2.000


# Data preparation

In [4]:
df_2019.drop('retailer_id', axis=1, inplace=True)
df_2020.drop('retailer_id', axis=1, inplace=True)
df_2021.drop('retailer_id', axis=1, inplace=True)

## Checking empty cells existence

In [5]:
df_2019.isnull().sum()

order_date         0
product_id         0
quantities_sold    0
dtype: int64

In [6]:
df_2020.isnull().sum()

order_date         0
product_id         0
quantities_sold    0
dtype: int64

In [7]:
df_2021.isnull().sum()

order_date         0
product_id         0
quantities_sold    0
dtype: int64

## Checking negative values or zero values existence

In [8]:
(df_2019['product_id'] <= 0).values.any(),(df_2019['quantities_sold'] <= 0).values.any()

(False, False)

In [9]:
(df_2020['product_id'] <= 0).values.any(),(df_2020['quantities_sold'] <= 0).values.any()

(False, False)

In [10]:
(df_2021['product_id'] <= 0).values.any(),(df_2021['quantities_sold'] <= 0).values.any()

(False, False)

# Add feature for season

In [12]:
def season_of_date(df, year):
    df['season'] = 'Season'
    year = str(year)

    seasons = {'Spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'Autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}


    df['season'] = df['order_date'].apply(lambda x: 'Spring' if x in seasons['Spring'] 
                                        else ('Summer' if x in seasons['Summer']
                                        else('Autumn' if x in seasons['Autumn'] else 'Winter')))
    

    return df

## Checking outliers existence

In [11]:
def check_outliers_existence(df):
    flag = False
    temp_frame = list()
    
    for i in df['product_id'].unique():
        product_frame = df[df['product_id'] == i]
        product_frame['quantity_std'] = (product_frame['quantities_sold'] - product_frame['quantities_sold'].mean()) / product_frame['quantities_sold'].std()
        count = 0
        count += len(product_frame['quantity_std'] == 3)
        count += len(product_frame['quantity_std'] == -3)
        temp_frame.append(count)
        
    if sum(temp_frame) != 0:
        flag = True
        
    return flag

## Drop outliers

In [12]:
def drop_outliers(df):
    frames_list = list()
    for i in df['product_id'].unique():
        product_frame = df[df['product_id'] == i]
        product_frame['quantity_std'] = (product_frame['quantities_sold'] - product_frame['quantities_sold'].mean()) / product_frame['quantities_sold'].std()
        product_frame1 = product_frame.drop(product_frame['quantity_std'].index[np.abs(product_frame['quantity_std']) > 3].tolist(), inplace=True)
        product_frame1 = product_frame.drop('quantity_std', axis=1,inplace=True)
        frames_list.append(product_frame)
    return pd.concat(frames_list)

## Resample data

In [13]:
def resampel_data(df, paramter):
    
    frames_list = list()
    for i in df['product_id'].unique():
        product_frame = df[df['product_id'] == i]
        temp_frame = product_frame.set_index('order_date').resample(paramter)['quantities_sold'].sum().reset_index()
        temp_frame.insert(1, 'product_id', i)
        frames_list.append(temp_frame.sort_values(by='order_date'))
    return pd.concat(frames_list)       

## Get common products

In [14]:
def get_common_products(df, products_ids):
    
    frames_list = list()
    for value in products_ids:
        frames_list.append(df[df['product_id'] == value].sort_values(by='order_date'))
    return pd.concat(frames_list)

## Get products with ids

In [15]:
def get_products_with_ids(df, products_ids):
    
    frames_list = list()
    
    for i in products_ids:
        temp_frame = df[df['product_id'] == i]
        temp_frame.sort_values(by='order_date')
        frames_list.append(temp_frame)
        
    return frames_list

# Data analysis
## KPI(s)
* Demand
* Number of products transactions

In [16]:
print('Number of products in year 2019 = {} products'.format(len(df_2019['product_id'].unique())))

Number of products in year 2019 = 921 products


In [17]:
print('Number of products in year 2020 = {} products'.format(len(df_2020['product_id'].unique())))

Number of products in year 2020 = 5041 products


In [18]:
print('Number of products in year 2021 = {} products'.format(len(df_2021['product_id'].unique())))

Number of products in year 2021 = 10290 products


## Get commen products IDs

In [16]:
common_products_ids = set(df_2019['product_id']) & set(df_2020['product_id']) & set(df_2021['product_id'])

In [17]:
print('Number of common products in all years = {} products'.format(len(common_products_ids)))

Number of common products in all years = 844 products


In [18]:
combined_df = pd.concat([df_2019, df_2020, df_2021], ignore_index=True)  

In [19]:
common_products = get_common_products(combined_df, common_products_ids)

### Daily data

In [ ]:
daily_df = resampel_data(common_products,'D')

In [ ]:
len(daily_df)

In [ ]:
print(check_outliers_existence(daily_df))

In [ ]:
new_daily_df = drop_outliers(daily_df)

In [ ]:
print('Outliers percentage = {}'.format((len(daily_df) - len(new_daily_df)) / len(daily_df))) 

### Weekly data

In [22]:
weekly_df = resampel_data(common_products,'W')

In [23]:
len(weekly_df)

94999

In [24]:
print(check_outliers_existence(weekly_df))

True


In [25]:
new_weekly_df = drop_outliers(weekly_df)

In [26]:
len(new_weekly_df)

93500

In [27]:
print('Outliers percentage = {}'.format((len(weekly_df) - len(new_weekly_df)) / len(weekly_df)))

Outliers percentage = 0.015779113464352255


### Monthly data

In [20]:
monthly_df = resampel_data(common_products,'M')

In [21]:
len(monthly_df)

22503

In [22]:
print(check_outliers_existence(monthly_df))

True


In [23]:
new_monthly_df = drop_outliers(monthly_df)

In [24]:
len(new_monthly_df)

22296

In [25]:
print('Outliers percentage = {}'.format((len(monthly_df) - len(new_monthly_df)) / len(monthly_df)))

Outliers percentage = 0.009198773496867084


# Plotting data one product (8)

In [ ]:
# fig = px.line(daily_df[daily_df['product_id'] == 8], x='order_date', y='quantities_sold', title='Daily data product id 8')
# fig.show()

In [ ]:
# fig = px.line(new_weekly_df[weekly_df['product_id'] == 8], x='order_date', y='quantities_sold', title='Weekly data product id 8')
# fig.show()

In [ ]:
# fig = px.line(new_monthly_df[monthly_df['product_id'] == 8], x='order_date', y='quantities_sold', title='Monthly data product id 8')
# fig.show()

# Transactions number

In [ ]:
fig = px.histogram(x=df_2019['product_id'].value_counts().values)
fig.update_layout(bargap=0.4)
fig.show()

In [ ]:
fig = px.histogram(x=df_2020['product_id'].value_counts().values)
fig.update_layout(bargap=0.4)
fig.show()

In [ ]:
fig = px.histogram(x=df_2021['product_id'].value_counts().values)
fig.update_layout(bargap=0.4)
fig.show()

## Show percentages of data

In [ ]:
size1 = len(get_common_products(combined_df, common_products_ids)) # size of commen products data
size2 = len(combined_df)  # size of the whole data

fig = px.pie(values=[size1, size2], names=['Common products data', 'Original data'])
fig.show()

In [ ]:
fig = px.bar(x=['2019' , '2020', '2021'],
             y=[len(df_2019['product_id'].unique()), len(df_2020['product_id'].unique()),len(df_2021['product_id'].unique())],
             title='Number of products sold each year',
             labels={'x': 'Year', 'y':'Number of products sold'})
fig.show()

## Show distributions of transactions

In [ ]:
counts = list(common_products['product_id'].value_counts().values)

fig = px.histogram(counts, nbins=20,title='Common products transactions before compression')
fig.update_layout(bargap=0.4)
fig.show()

In [ ]:
counts = list(daily_df['product_id'].value_counts().values)

fig = px.histogram(counts, nbins=20,title='Common products transactions after compression (daily)')
fig.update_layout(bargap=0.4)
fig.show()

In [ ]:
counts = list(new_weekly_df['product_id'].value_counts().values)

fig = px.histogram(counts, nbins=20,title='Common products transactions after compression (weekly)')
fig.update_layout(bargap=0.4)
fig.show()

In [ ]:
counts = list(new_monthly_df['product_id'].value_counts().values)

fig = px.histogram(counts, nbins=20,title='Common products transactions after compression (monthly)')
fig.update_layout(bargap=0.4)
fig.show()

## Get top ten products ids

In [ ]:
monthly_sum = (new_monthly_df.groupby(['product_id']).sum()).sort_values(by='quantities_sold', ascending=False)
monthly_sum[0:10]

## Create subplots

In [ ]:
size1 = len(get_common_products(combined_df, common_products_ids)) # size of commen products data
size2 = len(combined_df)  # size of the whole data

counts1 = list(common_products['product_id'].value_counts().values)
counts2 = list(daily_df['product_id'].value_counts().values)
counts3 = list(new_weekly_df['product_id'].value_counts().values)
counts4 = list(new_monthly_df['product_id'].value_counts().values)

fig = make_subplots(rows=9, cols=1,
                   specs=[[{'type': 'bar'}],
                          [{'type': 'bar'}],
                          [{'type': 'bar'}],
                          [{'type': 'bar'}],
                          [{'type': 'pie'}],
                          [{'type':'bar'}],
                          [{'type': 'bar'}],
                          [{'type':'bar'}],
                          [{'type': 'bar'}]],
                   subplot_titles=('Number of products sold each year',
                                   'Year 2019 products transactions',
                                   'Year 2020 products transactions',
                                   'Year 2021 products transactions',
                                   'Percentage of common products data',
                                   'Common products transactions before resampeling',
                                   'Common products transactions after resampeling (daily)',
                                   'Common products transactions after resampeling (weekly)',
                                   'Common products transactions after resampeling (monthly)'))

fig.add_trace(
    go.Bar(x=['2019' , '2020', '2021'],
             y=[len(df_2019['product_id'].unique()), len(df_2020['product_id'].unique()),len(df_2021['product_id'].unique())],
          text=[len(df_2019['product_id'].unique()), len(df_2020['product_id'].unique()),len(df_2021['product_id'].unique())]),
    row=1, col=1)

fig.update_xaxes(title_text='Year', row=1, col=1)
fig.update_yaxes(title_text='Number of products sold', row=1,col=1)

fig.add_trace(go.Histogram(x=df_2019['product_id'].value_counts().values, nbinsy=20), row=2, col=1)

fig.update_xaxes(title_text='Number of transactions', row=2, col=1)
fig.update_yaxes(title_text='Number of products', row=2,col=1)

fig.add_trace(go.Histogram(x=df_2020['product_id'].value_counts().values, nbinsy=20), row=3, col=1)

fig.update_xaxes(title_text='Number of transactions', row=3, col=1)
fig.update_yaxes(title_text='Number of products', row=3,col=1)

fig.add_trace(go.Histogram(x=df_2021['product_id'].value_counts().values, nbinsy=20), row=4, col=1)

fig.update_xaxes(title_text='Number of transactions', row=4, col=1)
fig.update_yaxes(title_text='Number of products', row=4,col=1)

fig.add_trace(
    go.Pie(values=[size1, size2], labels=['Common products data', 'Orignal data']),
    row=5, col=1)

fig.add_trace(go.Histogram(x=counts1, nbinsy=20),
             row=6, col=1)

fig.update_xaxes(title_text='Number of transactions', row=6, col=1)
fig.update_yaxes(title_text='Number of products', row=6, col=1)

fig.add_trace(go.Histogram(x=counts2, nbinsy=20),
             row=7, col=1)

fig.update_xaxes(title_text='Number of transactions', row=7, col=1)
fig.update_yaxes(title_text='Number of products', row=7, col=1)

fig.add_trace(go.Histogram(x=counts3, nbinsy=20),
             row=8, col=1)

fig.update_xaxes(title_text='Number of transactions', row=8, col=1)
fig.update_yaxes(title_text='Number of products', row=8, col=1)

fig.add_trace(go.Histogram(x=counts4, nbinsy=20),
             row=9, col=1)

fig.update_xaxes(title_text='Number of transactions', row=9, col=1)
fig.update_yaxes(title_text='Number of products', row=9, col=1)


fig.update_layout(height= 2000, width = 1350, title_text='Data summary', bargap=0.5)


fig.write_html('Data summary plots.html')

In [ ]:
fig = px.histogram(x=df_2019['product_id'].value_counts().values, nbins=20)
fig.update_xaxes(title_text='Number of transactions')
fig.update_yaxes(title_text='Number of products')
fig.update_layout(height= 600, width = 900, title_text='Year 2019 products transactions', bargap=0.5)
fig.write_html('year 2019 transactions.html')

In [ ]:
fig = px.histogram(x=df_2020['product_id'].value_counts().values, nbins=20)
fig.update_xaxes(title_text='Number of transactions')
fig.update_yaxes(title_text='Number of products')
fig.update_layout(height= 600, width = 900, title_text='Year 2020 products transactions', bargap=0.5)
fig.write_html('year 2020 transactions.html')

In [ ]:
fig = px.histogram(x=df_2021['product_id'].value_counts().values, nbins=20)
fig.update_xaxes(title_text='Number of transactions')
fig.update_yaxes(title_text='Number of products')
fig.update_layout(height= 600, width = 900, title_text='Year 2021 products transactions', bargap=0.5)
fig.write_html('year 2021 transactions.html')

## Plotting top ten products

In [ ]:
def plot_top_ten_products(df, products_ids, axis_name,name):
    
    fig = go.Figure()

    products_list = sorted(products_ids)

    for product_id in products_list:
        fig.add_trace(
            go.Scatter(
                x=df['order_date'][df['product_id'] == product_id],
                y=df['quantities_sold'][df['product_id'] == product_id],
                name=str(product_id), visible=True
            )
        )

    buttons = []

    for i, product_id in enumerate(products_list):
        args = [False] * len(products_list)
        args[i] = True

        button = dict(label=str(product_id),
                      method="update",
                      args=[{"visible": args}])

        buttons.append(button)

    fig.update_layout(
        xaxis_title=axis_name,
        yaxis_title='Quantity',
        updatemenus=[dict(active=0, type="dropdown", buttons=buttons, x=0, y=1.1, xanchor='left', yanchor='bottom')],
        autosize=False, width=1000, height=800, title_text=name
    )
    
    fig.write_html(name + '.html')

# Top ten products ids

In [ ]:
ids = [15, 3422, 16, 3705, 355, 2999, 3788, 3805, 4075, 3739]  

In [ ]:
plot_top_ten_products(daily_df, ids, 'Dyas', 'Top ten products daily data')

In [ ]:
plot_top_ten_products(new_weekly_df, ids, 'Weeks', 'Top ten products weekly data')

In [ ]:
plot_top_ten_products(new_monthly_df, ids, 'Months','Top ten products monthly data')

# Applying Exponential smoothing forecast on top ten products

## Monthly data

In [ ]:
top_ten_products_monthly_data = get_products_with_ids(new_monthly_df, ids)

## Weekly data

### Forecasting on weekly data is not applicable (multiplicative) becasue of there are enough number of zeros
### Because in multiplicative model for Holt Winter trend is calculated as division of two data points rather than subtraction (incase of additive model). Multiplicative model is generally more stable than additive model.
### So, we will apply (additive) on weekly data

In [ ]:
new_weekly_df.head(20)

In [ ]:
top_ten_products_weekly_data = get_products_with_ids(new_weekly_df, ids)

# Decompose top ten products

### matplotlib decompose

In [ ]:
def decompose_products_graphs(list_df, model, period, products_ids):
    
    for i in range(len(list_df)):
        graph = seasonal_decompose(list_df[i]['quantities_sold'], model=model, period=period)
        fig = graph.plot()
        fig.show()

In [ ]:
decompose_products_graphs(top_ten_products_weekly_data, 'additive', 4, ids)

### Plotly decompose

In [ ]:
def plot_seasonal_decompose(result:DecomposeResult, dates:pd.Series, title:'', time_scale:''):
    x_axis = dates
    return (
        make_subplots(
            rows=4,
            cols=1,
            subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residuals'],
        )
        .add_trace(
            go.Scatter(x=x_axis, y=result.observed, mode='lines', name='Observed'),
            row=1,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_axis, y=result.trend, mode='lines', name='Trend'),
            row=2,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_axis, y=result.seasonal, mode='lines', name='Seasonal'),
            row=3,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_axis, y=result.resid, mode='lines', name='Residual'),
            row=4,
            col=1,
        )
        .update_layout(
            height=900, title_text='Product id ' + title + time_scale + ' seasonal decomposition', margin={'t':100}, title_x=0.5
        )
    )

# Monthly decomposition

In [ ]:
for i in range(len(top_ten_products_monthly_data)):
    decomposition = seasonal_decompose(top_ten_products_monthly_data[i]['quantities_sold'], period=6)
    fig = plot_seasonal_decompose(decomposition, top_ten_products_monthly_data[i]['order_date'], str(top_ten_products_monthly_data[i]['product_id'].loc[0]), ' monthly')
    fig.update_layout(height=900, width=1100)
    fig.write_html('product id ' + str(top_ten_products_monthly_data[i]['product_id'].loc[0]) +' monthly decomposition' + '.html')

# Weekly decomposition

In [ ]:
for i in range(len(top_ten_products_weekly_data)):
    decomposition = seasonal_decompose(top_ten_products_weekly_data[i]['quantities_sold'], period=4)
    fig = plot_seasonal_decompose(decomposition, top_ten_products_weekly_data[i]['order_date'], str(top_ten_products_weekly_data[i]['product_id'].loc[0]), ' weekly')
    fig.update_layout(height=900, width=1100)
    fig.write_html('product id ' + str(top_ten_products_weekly_data[i]['product_id'].loc[0]) +' weekly decomposition' + '.html')

## Apply holt winter forecasting on top ten products

In [ ]:
def apply_holt_winter(list_df, products_ids, season, periods_num, frequency, axis_name,name):
    
    fig = make_subplots(rows=10, cols=1, subplot_titles=(['product id ' + str(i) for i in products_ids]))
                        
    model_info = dict()
    MAE_values = list()
    count = 1
    
    for frame in list_df:
        model = ExponentialSmoothing(frame['quantities_sold'], seasonal = season, seasonal_periods = periods_num)
        model.fit()
        
        model_forecast = model.predict(model.params,start = 0, end = len(frame) + 4)
        
        forecasted_frame = pd.DataFrame()
        forecasted_frame['order_date'] = pd.date_range(start =frame['order_date'][0], periods = len(frame) + 5, freq=frequency)
        forecasted_frame['quantities_sold'] = model_forecast
        
        MAE = float(np.sum(np.abs(frame['quantities_sold'] - forecasted_frame['quantities_sold'][:len(frame)]))) / len(frame)
        RMSE = (float(np.sum((frame['quantities_sold'] - forecasted_frame['quantities_sold'][:len(frame)])**2)) / len(frame))**0.5
        MAPE = ((np.sum(np.abs((frame['quantities_sold'] - forecasted_frame['quantities_sold'][:len(frame)]) / frame['quantities_sold']))) / len(frame)) / 100
        
        model_info['Product id ' + str(frame['product_id'][0]) + ' alpha value'] = model.params['smoothing_level']    # alpha
        model_info['Product id ' + str(frame['product_id'][0]) + ' beta value'] = model.params['smoothing_seasonal']  # beta
        model_info['Product id ' + str(frame['product_id'][0]) + ' MAE value'] = MAE
        model_info['Product id ' + str(frame['product_id'][0]) + ' RMSE value'] = RMSE
        model_info['Product id ' + str(frame['product_id'][0]) + ' MAPE value (%)'] = MAPE
        
        MAE_values.append(MAE)
        
        fig.add_trace(go.Scatter(x=frame['order_date'], y=frame['quantities_sold'], name='Actual'),row=count, col=1)
        fig.add_trace(go.Scatter(x=forecasted_frame['order_date'], y=forecasted_frame['quantities_sold'], name='Forecast'),row=count, col=1)
        
        fig.update_xaxes(title_text=axis_name, row=count, col=1)
        fig.update_yaxes(title_text='Quantity', row=count, col=1)
        count += 1
        
    fig.update_layout(height=3000, width=1350, title_text='Top ten products forecating on monthly data')


    fig.write_html(name + ' forecast.html')
    
    return model_info, MAE_values

### Monthly

In [ ]:
from IPython.display import Image
Image('Screenshot.png')

### Weekly

In [ ]:
info, weekly_error_values = apply_holt_winter(top_ten_products_weekly_data, ids, 'additive', 4, 'W', 'Weeks','Weekly top ten products')

for item in info.items():
    print(item)  

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('MAE values resulted from monthly forecast',
                                                    'MAE values resulted from weekly forecast'))

fig.add_trace(go.Histogram(x=monthly_error_values, name='Monthly'),row=1, col=1)

fig.update_xaxes(title_text='Value of MAE', row=1, col=1)
fig.update_yaxes(title_text='Number of products', row=1, col=1)

fig.add_trace(go.Histogram(x=weekly_error_values, name='Weekly'),row=1, col=2)

fig.update_xaxes(title_text='Value of MAE' , row=1, col=2)
fig.update_yaxes(title_text='Number of products', row=1, col=2)


fig.update_layout(height=600, width=1300, title_text='MAE values', bargap=0.5)

fig.write_html('MAE values.html')

# Apply time series regression

## On weekly df

In [ ]:
def apply_random_forest_regression(df, estimatores_num):
    
    frames_list = list()
    for i in df['product_id'].unique():
        info = dict()
        model = RandomForestRegressor(n_estimators=estimatores_num, random_state=42)
        product_frame = df[df['product_id'] == i]
        
        temp_frame = season_of_date(product_frame[(product_frame['order_date'] >= '01/01/2019') & (product_frame['order_date'] <= '31/12/2019')],2019)
        temp_frame1 = season_of_date(product_frame[(product_frame['order_date'] >= '01/01/2020') & (product_frame['order_date'] <= '31/12/2020')],2020)
        temp_frame2 = season_of_date(product_frame[(product_frame['order_date'] >= '01/01/2021') & (product_frame['order_date'] <= '31/12/2021')],2021)

        product_frame = pd.concat([temp_frame, temp_frame1, temp_frame2], ignore_index=True)
        
        product_frame['order_year'] = product_frame['order_date'].dt.year
        product_frame['order_month'] = product_frame['order_date'].dt.month
        product_frame['order_day'] = product_frame['order_date'].dt.day

        dummy = pd.get_dummies(product_frame['season'])
        dummy.drop('Winter', axis=1, inplace=True)

        product_frame = pd.concat([product_frame, dummy], axis=1)
        product_frame.drop(['order_date', 'product_id', 'season'], axis=1, inplace=True)

        # 60% train, 20% validation, 20% test 
        train, validate, test = np.split(product_frame.sample(frac=1), [int(.6*len(product_frame)), int(.8*len(product_frame))])
        
        X_train, y_train = train.drop('quantities_sold', axis=1), train['quantities_sold']  
        X_valid, y_valid = validate.drop('quantities_sold', axis=1), validate['quantities_sold'] 
        X_test, y_test = test.drop('quantities_sold', axis=1), test['quantities_sold'] 
        
        model.fit(X_train, y_train)
        
        train_preds = model.predict(X_train)
        val_preds = model.predict(X_valid)
        tes_preds = model.predict(X_test)

        info['product_id'] = [i]
        info['training size (60%)'] = [len(train)]
        info['training MAE'] = [mean_absolute_error(y_train, train_preds)]
        info['training RMSLE'] = [np.sqrt(mean_squared_log_error(y_train, train_preds))]
        info['training r2'] = [r2_score(y_train, train_preds)]

        info['validation size (20%)'] = [len(validate)]
        info['vaild MAE'] = [mean_absolute_error(y_valid, val_preds)]
        info['valid RMSLE'] = [np.sqrt(mean_squared_log_error(y_valid, val_preds))]
        # info['valid r2'] = [r2_score(y_valid, val_preds)]

        info['test size (20%)'] = [len(test)]
        info['test MAE'] = [mean_absolute_error(y_test, tes_preds)]
        info['test RMSLE'] = [np.sqrt(mean_squared_log_error(y_test, tes_preds))]
        # info['test r2'] = [r2_score(y_test, tes_preds)]

        frames_list.append(pd.DataFrame(info))
    
    pd.concat(frames_list, ignore_index=True).to_excel('random forest regression with '+ str(estimatores_num) +' estimators.xlsx')


In [31]:
test = new_monthly_df.copy()

test = pd.concat([season_of_date(test[(test['order_date'] > '01/01/2019') & (test['order_date'] <= '12/31/2019')],2019),
                  season_of_date(test[(test['order_date'] > '01/01/2020') & (test['order_date'] <= '12/31/2020')],2020),
                  season_of_date(test[(test['order_date'] > '01/01/2021') & (test['order_date'] <= '12/31/2021')],2021)], ignore_index=True)
test = test.sort_values(by='product_id')
test.to_csv('monthly data.csv')

In [ ]:
test = new_weekly_df.copy()

test = pd.concat([season_of_date(test[(test['order_date'] > '01/01/2019') & (test['order_date'] <= '12/31/2019')],2019),
                  season_of_date(test[(test['order_date'] > '01/01/2020') & (test['order_date'] <= '12/31/2020')],2020),
                  season_of_date(test[(test['order_date'] > '01/01/2021') & (test['order_date'] <= '12/31/2021')],2021)], ignore_index=True)
test = test.sort_values(by='product_id')
test.to_csv('weekly data.csv')

In [32]:
test.head()

,order_date,product_id,quantities_sold,season
12513,2021-08-31,8,6559.000,Summer
12515,2021-10-31,8,8212.500,Autumn
12514,2021-09-30,8,6793.000,Autumn
2451,2020-12-31,8,3107.330,Winter
12512,2021-07-31,8,5704.010,Summer


# (testing) apply linear regression on each product sparatedly

In [6]:
df = pd.read_csv('weekly data.csv')
df['order_date'] = pd.to_datetime(df['order_date'])
df.drop('season', axis=1, inplace=True)

df.head()

,order_date,product_id,quantities_sold
0,2021-01-03,8,569.000
1,2020-04-19,8,94.000
2,2020-04-12,8,318.000
3,2020-04-05,8,124.000
4,2020-03-29,8,156.000


In [8]:
cor_list = list()
list_cor = list()
for i in df['product_id'].unique():
    info = dict()
    temp_f = df[df['product_id'] == i]

    temp_f['date'] = pd.to_numeric(temp_f['order_date'])
    temp_f['target'] = temp_f['quantities_sold']

    temp_f.drop(['order_date', 'product_id', 'quantities_sold'], axis=1, inplace=True)
    cor = temp_f.corr()

    info['product_id'] = [i]
    info['correlation with date'] = ['{:.2f}'.format(cor['date'][1])]
    
    if np.abs(cor['date'][1]) < 0.50:
        df.drop(df[df['product_id'] == i].index, inplace=True)
        
    list_cor.append(cor['date'][1])
    cor_list.append(pd.DataFrame(info))
pd.concat(cor_list, ignore_index=True).to_csv('correlation with date.csv', index=False)

In [ ]:
flag = ''
for i in df['product_id'].unique():
    te = df[df['product_id'] == i]
    te.drop('product_id', axis=1, inplace=True)

    fig = px.scatter(te, x='order_date', y='quantities_sold', title='product ' + str(i))
    fig.show()

    flag = input('continue y/n')

    if flag == 'n':
        break



In [32]:
from sklearn.preprocessing import StandardScaler
tx = df[df['product_id'] == 15]
tx = tx.sort_values(by='order_date')
tx.drop('product_id', axis=1, inplace=True)

tx['order_date'] = pd.to_numeric(tx['order_date'])
sc = StandardScaler()
u = sc.fit_transform(np.array(tx['order_date']).reshape(-1, 1))


tx['order_date'] = u
tx.head()

,order_date,quantities_sold
818,-1.714,30.000
808,-1.685,8.000
809,-1.656,7.000
810,-1.628,8.000
811,-1.599,15.000


In [33]:
test_set = tx.tail(8)
tx.drop(tx.tail(8).index, inplace=True)

len(tx)

112

In [34]:
X_train = tx.drop('quantities_sold', axis=1)
y_train = tx['quantities_sold']

X_test = test_set.drop('quantities_sold', axis=1)
y_test = test_set['quantities_sold']

len(X_train), len(X_test), len(y_train), len(y_test)

(112, 8, 112, 8)

In [22]:
from sklearn.linear_model import LinearRegression

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import statsmodels.api as sm

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [35]:
lin = LinearRegression()
lin.fit(X_train, y_train)

LinearRegression()

In [36]:
lin.score(X_train, y_train)

0.6076890395794718

In [ ]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

In [38]:
y_pred = lin.predict(X_test)

mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred)

(1732.4165124933716, 3327402.0323536457)

In [39]:
lin.score(X_test, y_test)

-0.14391798897160601

In [ ]:
error = y_test - y_pred

fig = px.histogram(error, nbins=20,title='Weekly MAE (Linear regression)', text_auto=True)
fig.update_layout(bargap=0.4)
fig.update_xaxes(title_text='MAE')
fig.update_yaxes(title_text='Count')
fig.show()